# Курсовая работа по дисциплине "Структуры и алгоритмы обработки данных"
## Выполнила студентка группы БВТ2103 Полуян Юлия Александровна

### Задание

Написать программу, которая будет считывать данные из CSV файла, содержащего
информацию о продажах товаров в магазине. Данные в файле содержатся в следующем
формате:
| Номер заказа | Дата заказа | Название товара | Категория товара | Количество продаж | Цена
за единицу | Общая стоимость |

Необходимо:
1. Рассчитать общую выручку магазина.
2. Найти товар, который был продан наибольшее количество раз.
3. Найти товар, который принес наибольшую выручку.
4. Составить отчет, содержащий информацию об общей выручке магазина, количестве проданных единиц каждого товара и доле каждого товара в общей выручке.

Для решения задач необходимо использовать структуры данных, такие как массивы и хеш-таблицы, а также различные алгоритмы обработки данных, например, сортировку и поиск. Также необходимо учитывать возможные ошибки ввода-вывода и обрабатывать их в
соответствии с требованиями.

In [33]:
class ValueError(Exception): #при ошибочных значениях поступивших на вход данных
    
    def __init__(self, message):
        super().__init__(f"Error in: \n{message}")

def check_values(position):
    #проверка на положительные числовые значения
    if (int(position["Номер заказа"]) < 0 and int(position["Количество продаж"]) < 0 
        and int(position["Цена за единицу"]) < 0 and int(position["Общая стоимость"]) < 0):
        return False
    #проверка формата даты - день, месяц, год
    date = position["Дата заказа"]
    if 1 > int(date[0]) > 31 and 1 > int(date[1]) > 12 and int(date[2]) > 2024:
      return False
    return True


class KeysNoEqualError(Exception): #при несоответствии ключей таблицы требованиям в условии

    def __init__(self, keys, message = "Keys don't match!"):
        self.keys = keys
        self.message = message
        super().__init__(self.message)


check_keys = [
        "Номер заказа", 
        "Дата заказа",
        "Название товара",
        "Категория товара",
        "Количество продаж",
        "Цена за единицу",
        "Общая стоимость"
    ]

In [50]:
import csv

def read_csv(file_path):   
 
    with open(file_path, encoding='utf-8') as file: #преобразование элементов таблицы в словарь
        file_reader = csv.DictReader(file, delimiter = ";")
        positions = []
        for row in file_reader:
            if not check_values(row):
                raise ValueError
            positions.append(row)

    with open('table.csv', 'r', encoding='utf-8', newline='') as csvfile: #получение списка ключей (названий столбцов) таблицы
        spamreader = csv.reader(csvfile, delimiter=';')
        for row in spamreader:
            keys = row
            break

    if keys != check_keys: #обработка исключения
        raise KeysNoEqualError(keys)
    
    return positions, keys

positions, keys = read_csv("table.csv")


### Задание 1

In [35]:
sum = 0

for item in positions:
  try:
    sum += int(item["Общая стоимость"])
  except ValueError:
    print("Data not a number")
    break 

print("Общая выручка:", sum)

Общая выручка: 241969500


### Задание 2

In [36]:
def quick_sort(arr, key):
  if len(arr) <= 1:
    return arr

  prop = int(arr[len(arr) // 2][key])
  right_part = [i for i in arr if int(i[key]) > prop]
  left_part = [i for i in arr if int(i[key]) < prop]
  middle = [i for i in arr if int(i[key]) == prop] 

  return quick_sort(left_part, key) + middle + quick_sort(right_part, key)

In [37]:
quick_sort(positions, "Количество продаж")[-1]

{'Номер заказа': '4010',
 'Дата заказа': '10.02.2022',
 'Название товара': 'Philips Hue Smart Bulbs',
 'Категория товара': 'Умный дом',
 'Количество продаж': '500',
 'Цена за единицу': '2990',
 'Общая стоимость': '1495000'}

### Задание 3

In [38]:
class Deque:

    def __init__(self): #инициализация
        self.positions = []

    def is_empty(self): #проверка на пустоту
        return self.positions == []
    
    def add_left(self, position): #добавление нового элемента в начало
        self.positions.insert(0,position)

    def add_right(self, position): #добавление нового элемента в конец
        self.positions.append(position)

    def remove_right(self): #извлечение элемента из конца
        return self.positions.pop()

    def remove_left(self): #извлечение элемента из начала
        return self.positions.pop(0)
    
    def get(self): #возвращение последнего элемента
        return self.positions[len(self.positions) - 1]

    def size(self): #возвращение количества элементов
        return len(self.positions)

In [39]:
def max_sort_with_deque(arr, key): #сортировка по возрастанию с использованием двух Деков
    deque1 = Deque()
    deque2 = Deque()

    for i in arr:
        deque1.add_right(i)

    while not deque1.is_empty():
        current = deque1.remove_right()
        while not deque2.is_empty() and int(deque2.get()[key]) > int(current[key]):
            deque1.add_left(deque2.remove_right())
        deque2.add_right(current)

    return deque2.get()

In [40]:
max_sort_with_deque(positions, "Общая стоимость")

{'Номер заказа': '4001',
 'Дата заказа': '01.02.2022',
 'Название товара': 'Apple iPhone 14',
 'Категория товара': 'Смартфоны',
 'Количество продаж': '500',
 'Цена за единицу': '89990',
 'Общая стоимость': '44995000'}

### Задание 4

In [51]:
print("Общая выручка:", sum)

for position in quick_sort(positions, "Общая стоимость"):
    try:
        print(
            "Название товара - ", position["Название товара"],";",
            "Количество продаж - ", position["Количество продаж"],";",
            "Доля товара в общей выручке - ", int(position["Общая стоимость"]) / sum * 100, "%"
        )   
    except KeysNoEqualError:
        print("Keys are wrong!")
        break


Общая выручка: 241969500
Название товара -  Philips Hue Smart Bulbs ; Количество продаж -  500 ; Доля товара в общей выручке -  0.6178464641204781 %
Название товара -  Philips Sonicare DiamondClean ; Количество продаж -  200 ; Доля товара в общей выручке -  0.8257239032192073 %
Название товара -  Nest Learning Thermostat ; Количество продаж -  100 ; Доля товара в общей выручке -  0.8261371784460437 %
Название товара -  Bose SoundLink Revolve+ ; Количество продаж -  100 ; Доля товара в общей выручке -  1.032774791864264 %
Название товара -  Bose QuietComfort 55 ; Количество продаж -  100 ; Доля товара в общей выручке -  1.446050018700704 %
Название товара -  DJI Mavic 3 ; Количество продаж -  25 ; Доля товара в общей выручке -  1.5496787818299413 %
Название товара -  Samsung Galaxy Watch 5 ; Количество продаж -  150 ; Доля товара в общей выручке -  1.8591186079237259 %
Название товара -  LG UltraFine ; Количество продаж -  50 ; Доля товара в общей выручке -  2.0661694965687825 %
Названи

In [54]:
print("Общая выручка:", sum)

# реализация в табличном виде через Pandas
import pandas as pd
positions = pd.read_csv('table.csv', sep=';')

def revenue_share(number):
    return str(number * 100 / sum) + '%'

positions['Доля выручки'] = positions['Общая стоимость'].apply(revenue_share)
positions[['Название товара', 'Количество продаж', 'Доля выручки']]



Общая выручка: 241969500


,Название товара,Количество продаж,Доля выручки
0,Apple iPhone 14,500,18.595318831505622%
1,Samsung Galaxy S22 Ultra,300,12.397016979412694%
2,Игровые приставки,200,4.958476171583609%
3,LG OLED TV,150,11.15781121174363%
4,Apple MacBook Pro 16,100,8.265091261501967%
5,Canon EOS R7,50,5.165733697842083%
6,Bose QuietComfort 55,100,1.446050018700704%
7,Samsung Galaxy Tab S8,75,2.169384984471183%
8,DJI Mavic 3,25,1.5496787818299413%
9,Philips Hue Smart Bulbs,500,0.617846464120478%
